# Object Following - Live Demo for Mecanum

このサンプルは、メカナムホイールを活用したオブジェクトトラッキングを行うデモプログラムです。  
平行移動と旋回を駆使して、特定の物体を追従します。  
本サンプルの実行には、[COCO dataset](http://cocodataset.org) が必要です。   
下記のコードを順番に実行してください。

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

In [ ]:
detections = model(camera.value)

print(detections)

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

In [ ]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
from jetbot import Robot

robot = Robot()

下記のコードを実行する際は、VS-C3のL・Rボタンのいずれかを押した状態で実行してください。
ロボットが急に動き出す可能性があります。

In [ ]:
from jetbot import bgr8_to_jpeg

blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, description='speed')
turn_speed_widget = widgets.FloatSlider(value=0.2, min=0.0, max=2.0, description='turn speed')
translation_speed_widget = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, description='translation speed')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_speed_widget,
    translation_speed_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    blocked_widget.value = prob_blocked
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    
        
    # otherwise go forward if no target detected
    if det is None:
        # turn left if blocked
        if prob_blocked > 0.5:
            robot.left(turn_speed_widget.value)
            image_widget.value = bgr8_to_jpeg(image)
            return
        else:
            robot.forward(float(speed_widget.value))
        
    # otherwsie steer towards target
    else:
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(det)
        point = det['bbox']
        if point[3] > 0.5:
            if abs(center[0]) > 0.08:
                robot.set_motors(
                    float(turn_speed_widget.value / abs(center[0]) * center[0]),
                    float(-turn_speed_widget.value / abs(center[0]) * center[0]),
                    float(turn_speed_widget.value / abs(center[0]) * center[0]),
                    float(-turn_speed_widget.value / abs(center[0]) * center[0])
                    )
            elif point[3]>0.6:
                robot.backward(float(speed_widget.value))
            else:
                robot.stop()
        elif abs(center[0]) > 0.08:
            if center[0]<0:
                robot.leftTranslation(float(translation_speed_widget.value))
            else:
                robot.rightTranslation(float(translation_speed_widget.value))
        else:
            robot.forward(float(speed_widget.value))
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

下記のコードを実行したのち、VS-C3のボタンから指を離してください。

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

ロボットを停止する際には下記のコードを実行してください。

In [ ]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()